<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/IBM_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain
%pip install langchain_chroma
%pip install langchain-community
%pip install -U langchain_ibm
%pip install unstructured
%pip install "ibm-watson-machine-learning>=1.0.327"
%pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.2 MB/s eta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.7 MB/s eta 0:00:00


In [75]:
import os
import getpass

from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

from langchain_community.document_loaders import UnstructuredURLLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.schema import Document

from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain


In [2]:
model_id = "ibm/granite-3-3-8b-instruct"
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ··········


In [74]:
chat_history = []

In [3]:
parameters = {
    GenParams.DECODING_METHOD: 'greedy',
    GenParams.TEMPERATURE: 2,
    GenParams.TOP_P: 0,
    GenParams.TOP_K: 100,
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.MAX_NEW_TOKENS: 512,
    GenParams.REPETITION_PENALTY:1.2,
    GenParams.STOP_SEQUENCES:['B)','\n'],
    GenParams.RETURN_OPTIONS: {'input_tokens': True,'generated_tokens': True, 'token_logprobs': True, 'token_ranks': True, }
}

In [4]:
llm = WatsonxLLM(
    model_id=model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id="62406522-8277-4f46-920c-aa5d5d2fb0c2",
    params=parameters
)

In [5]:
def build_vector_store(text: str, embeddings=None, persist_dir: str = "faiss_store") -> FAISS:
    """
    Split text, embed, and create FAISS vector store.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = splitter.split_text(text)
    # Wrap into Document objects
    documents = [Document(page_content=chunk) for chunk in docs]

    # Initialize embeddings (using WatsonxEmbeddings as placeholder)
    embeddings = embeddings or WatsonxEmbeddings(url=credentials.get("url"), apikey=credentials.get("apikey"), project_id="62406522-8277-4f46-920c-aa5d5d2fb0c2", model_id="ibm/granite-embedding-107m-multilingual")
    store = FAISS.from_documents(documents, embeddings)
    store.save_local(persist_dir)
    return store

In [90]:
def make_qa_chain(vector_store: FAISS):
    retriever = vector_store.as_retriever(search_kwargs={"k": 4})
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        verbose=True
    )
    return qa

In [8]:
text = """
So, now let's try to get a little bit of a sense of how this technology works. So, we're going to give a very simple introduction to the basic concepts that we will then dive into in more depth subsequently. So, one of the key things to Machine Learning is the idea that we're going to teach a machine to learn. So, the way that we do that, is we give the machine examples, and the examples are characterized by data samples and then what we would like the machine to predict of that data. So here is an example where the data is just a set of numbers or a vector of numbers, and then the label is what we would like to assign to that data. Here we're going to say that the label is denoted by y, here for simplicity, we're going to say that the label is either zero or one, but we can generalize that to other labels. So, as an example we might wish to analyze images and determine whether someone has a malignancy or not based upon what is seen in the image. So, in that case, the acts or the data would correspond to the pixellated image and then the label y which here would again be binary, would classify whether the image is malignant or not. So, the way that we teach a machine to learn, and by machine we mean algorithm, is we give it examples. So, we give it many examples here N examples, where an example corresponds to a data sample, and then the true label. So, here we have N examples where we have the data x, and the label y. Then what we would like to do is to take this data which we call training data to teach a machine or an algorithm to do prediction accurately in the sense that it is capable of reproducing the training data, in the sense that if you give it any of these vectors acts, it accurately predicts the associated y. But more importantly we would like it to be able to make predictions for future examples. So we would like to give it a new data sample x and predict the associated y. To make this a little bit concrete so that it's clear what we're talking about, let's consider a simple example. So, let's ask the question whether it is going to rain or not on a given day. So, let's assume that we can measure some characteristics of the environment at a given point in time for example, in the morning, we'd look at the level of cloud cover, the level of humidity, temperature, air pressure, et cetra. These are a set of numbers, those are the data and then on a given day, it either rains or not, we'll say that the y is equal to one if it rained, and y is equal to zero if it did not rain. So, the idea is that we would have a training set which would be composed of multiple examples of the associated data which is represented by the vector x, and the associated label which tells us whether it rained or not that day. We would then like to build a predictive model which takes in the training data, so, the training data is on the top left, it's all of the examples that we have based upon historical data of the associated conditions which are represented by x, and then y whether it rained or not that day. We have a mathematical model which we'll call the machine, that mathematical model has parameters which are denoted by the question mark, those are unknown parameters. What we would like to do is to learn those model parameters such that the algorithm or the mathematical model does an accurate job of representing the training set. Then ideally if we give that mathematical model a new example of conditions represented by the vector x, we would like that model to accurately predict whether it will rain that day. So, then the key task then would be how well the model performs when testing, and so in this case again, we would observe a new set of data x and then our goal is to predict the label y. So, this gives a very early introduction into Machine Learning. The key thing to take away from this is that we have training examples which correspond to data and outcomes that we would like to model to predict. Based upon that training data we then apply that training data to a model, that model is typically characterized by a set of parameters. What our goal is to try to learn the parameters of that model such that they are consistent with the training data. After we do that learning, we then apply that model to new data and make predictions in the future. What we will now do as we move forward is to try to dig into some of these details to provide more clarity on the aspects of Machine Learning.
"""

In [89]:
print("Building knowledge base...")
store = build_vector_store(text)
qa_chain = make_qa_chain(store)

Building knowledge base...


In [ ]:
query = "What is Machine Learning"

In [ ]:
result = qa_chain.invoke({"question": query, "chat_history": chat_history})
print("\nAnswer:", result['answer'])
chat_history.append((query, result['answer']))

In [29]:
query = "What is my previous question?"
result = qa_chain.invoke({"question": query, "chat_history": chat_history})
print("\nAnswer:", result)


Answer: {'question': 'What is my previous question?', 'chat_history': [('What is Machine Learning', "  Machine Learning is a field of study focused on developing algorithms that enable machines to learn patterns from data and make predictions or decisions without being explicitly programmed. It involves providing the machine with examples consisting of data samples (features) and corresponding desired outputs (labels). By analyzing numerous examples during a process called 'training', the algorithm learns to map input data to appropriate output labels. Once trained, the model can make predictions on unseen data using its learned pattern recognition capabilities. In essence, Machine Learning aims to create models that automatically improve their performance over time through experience, enabling them to tackle complex tasks such as image classification, natural language processing, and decision-making systems.")], 'answer': ' \n\nYour previous question was about understanding the conce

In [30]:
print("\nAnswer:", result['answer'])


Answer:  

Your previous question was about understanding the concept of Machine Learning using the example of predicting whether it will rain on a given day based on certain environmental measurements (data) and historical records (training set). You were interested in knowing how a model learns from this training data to make predictions for new, unseen data samples. Additionally, there was curiosity regarding the role of parameters within the model and their optimization process during the learning phase.



# Web deployment

In [80]:
from flask import Flask, request, jsonify, render_template_string, Response
from threading import Thread
from google.colab.output import eval_js
import time

app = Flask(__name__)

In [81]:
@app.route("/load_transcript", methods=["POST"])
def load_transcript():
    global qa_chain, chat_history
    chat_history = []
    transcript = request.json["transcript"]
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = [Document(page_content=chunk) for chunk in splitter.split_text(transcript)]

    embeddings = WatsonxEmbeddings(
        url=credentials.get("url"),
        apikey=credentials.get("apikey"),
        project_id="62406522-8277-4f46-920c-aa5d5d2fb0c2",
        model_id="ibm/granite-embedding-107m-multilingual"
    )

    vectorstore = FAISS.from_documents(docs, embeddings)
    qa_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())
    return jsonify({"message": "Transcript loaded and vector store built."})

In [82]:
@app.route("/ask", methods=["POST"])
def ask_question():
    query = request.json["question"]
    result = qa_chain({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    return jsonify({"answer": result["answer"]})

In [83]:
@app.route('/')
def index():
    html = '''<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Coursera AI Assistant</title>
  <link rel="icon" href="data:,">
  <style>
    body { font-family: Arial, sans-serif; margin: 40px; }
    textarea, input[type=text] {
      width: 100%; padding: 10px; margin-top: 10px; font-size: 16px;
      box-sizing: border-box;
    }
    button {
      padding: 10px 20px; font-size: 16px; margin-top: 10px;
      cursor: pointer;
    }
    #response {
      margin-top: 20px; white-space: pre-wrap;
      background: #f9f9f9; padding: 10px; border-radius: 5px;
      min-height: 50px;
    }
    .loading { color: #666; font-style: italic; }
  </style>
</head>
<body>
  <h1>🎓 Coursera AI Assistant</h1>

  <h3>Step 1: Paste Transcript</h3>
  <textarea id="transcript" rows="8" placeholder="Paste transcript here..."></textarea>
  <button id="load-btn">Load Transcript</button>

  <h3>Step 2: Ask a Question</h3>
  <input type="text" id="question" placeholder="Type your question here..." />
  <button id="ask-btn">Ask</button>

  <div id="response"></div>

  <script>
    console.log('Script loaded');

    document.addEventListener('DOMContentLoaded', function() {
      console.log('DOM loaded');

      const loadBtn = document.getElementById('load-btn');
      const askBtn = document.getElementById('ask-btn');
      const respDiv = document.getElementById('response');

      if (!loadBtn || !askBtn || !respDiv) {
        console.error('Elements not found');
        return;
      }

      loadBtn.addEventListener('click', async function() {
        console.log('Load button clicked');
        const transcript = document.getElementById('transcript').value;

        if (!transcript.trim()) {
          alert('Please enter a transcript first');
          return;
        }

        try {
          respDiv.innerHTML = '<span class="loading">Loading transcript...</span>';

          const res = await fetch('/load_transcript', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ transcript: transcript })
          });

          const data = await res.json();
          respDiv.innerHTML = '';
          alert(data.message || 'Transcript loaded successfully!');
        } catch (error) {
          console.error('Error:', error);
          respDiv.innerHTML = 'Error loading transcript: ' + error.message;
        }
      });

      askBtn.addEventListener('click', async function() {
        console.log('Ask button clicked');
        const question = document.getElementById('question').value;

        if (!question.trim()) {
          alert('Please enter a question first');
          return;
        }

        try {
          respDiv.innerHTML = '<span class="loading">Generating answer...</span>';

          const res = await fetch('/ask', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ question: question })
          });

          const data = await res.json();
          respDiv.innerHTML = '💬 Answer:\\n' + (data.answer || 'No answer received');
        } catch (error) {
          console.error('Error:', error);
          respDiv.innerHTML = 'Error getting answer: ' + error.message;
        }
      });

      console.log('Event listeners attached');
    });
  </script>
</body>
</html>'''

    return Response(
        html,
        mimetype='text/html',
    )

In [84]:
def run():
    app.run(host="0.0.0.0", port=5055)

thread = Thread(target=run)
thread.start()

# Get the proxy URL
time.sleep(2)  # wait for Flask to start
print("Your app is running at this URL:")
print(eval_js("google.colab.kernel.proxyPort(5055)"))

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5055
 * Running on http://172.28.0.12:5055
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an a

Your app is running at this URL:


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


https://5055-m-s-1rt3nts46t83l-a.us-east4-0.prod.colab.dev
